In [1]:
#Import required libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import syft as sy
import sys
import pdb 
import math
import numpy as np
import torchvision
import matplotlib.pyplot as plt
import torchvision.models as models
from random import shuffle
from torch.utils.data import SubsetRandomSampler, WeightedRandomSampler
import torchvision.models as models
import datetime
import torch.nn.init as init
import torch

W0302 08:58:36.886226 10904 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was 'c:\users\florian\appdata\local\programs\python\python37\lib\site-packages\tf_encrypted-0.5.9-py3.7.egg\tf_encrypted/operations/secure_random/secure_random_module_tf_1.15.0-rc3.so'
W0302 08:58:36.918214 10904 module_wrapper.py:139] From c:\users\florian\appdata\local\programs\python\python37\lib\site-packages\tf_encrypted-0.5.9-py3.7.egg\tf_encrypted\session.py:24: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [2]:
torch.set_default_tensor_type(torch.cuda.FloatTensor)
use_cuda = True
kwargs = {'num_workers': 4, 'pin_memory': True} if use_cuda else {}
device = torch.device("cuda" if use_cuda else "cpu")
batch_size = 512
np.random.seed(1)
torch.manual_seed(1)
torch.set_num_threads(4)
backdoorType = "backdoor_green_0_5_percent"

# Creating workers

In [3]:
hook = sy.TorchHook(torch)  # <-- NEW: hook PyTorch ie add extra functionalities to support Federated Learning

alice = sy.VirtualWorker(hook, id="alice")  # <-- NEW: and alice
bob = sy.VirtualWorker(hook, id="bob")  # <-- NEW: define remote worker 
chalie = sy.VirtualWorker(hook, id="chalie")  # <-- NEW: define remote worker
dave = sy.VirtualWorker(hook, id="dave")  # <-- NEW: define remote worker
#evelyn = sy.VirtualWorker(hook, id="evelyn")  # <-- NEW: define remote worker
#a6= sy.VirtualWorker(hook, id="a6")  # <-- NEW: define remote worker
#a7= sy.VirtualWorker(hook, id="a7")  # <-- NEW: define remote worker
#a8= sy.VirtualWorker(hook, id="a8")  # <-- NEW: define remote worker
#a9= sy.VirtualWorker(hook, id="a9")  # <-- NEW: define remote worker
#a10= sy.VirtualWorker(hook, id="a10")  # <-- NEW: define remote worker
#a11= sy.VirtualWorker(hook, id="a11")  # <-- NEW: define remote worker
#a12= sy.VirtualWorker(hook, id="a12")  # <-- NEW: define remote worker
#a13= sy.VirtualWorker(hook, id="a13")  # <-- NEW: define remote worker
#a14= sy.VirtualWorker(hook, id="a14")  # <-- NEW: define remote worker
#a15= sy.VirtualWorker(hook, id="a15")  # <-- NEW: define remote worker
#a16= sy.VirtualWorker(hook, id="a16")  # <-- NEW: define remote worker
#a17= sy.VirtualWorker(hook, id="a17")  # <-- NEW: define remote worker
#a18= sy.VirtualWorker(hook, id="a18")  # <-- NEW: define remote worker
#a19= sy.VirtualWorker(hook, id="a19")  # <-- NEW: define remote worker


fraudulin = sy.VirtualWorker(hook, id="fraudulin")
#fraudrich = sy.VirtualWorker(hook, id="fraudrich") 

compute_nodes = [alice, bob, chalie, dave]
frauds = [fraudulin]

# Helper function for dataset loader generation

In [4]:
#def generateLoadersPerClass(dataset):
#    #loaders per class
#    loaders_per_class = []
#    for class_name in dataset.classes:
#        # get the indices in the dataset that are relative to that class
#        idx = [
#            pos for pos, item in enumerate(dataset.samples)
#            if item[1] == dataset.class_to_idx[class_name]]
#        # construct the corresponding dataloader thanks to a SubsetRandomSampler
#        loaders_per_class += [torch.utils.data.DataLoader(
#            dataset, 
#            batch_size=batch_size,
#            sampler=SubsetRandomSampler(idx),
#            **kwargs)]
#    return loaders_per_class

# Loading training & test datasets

In [5]:
data_transform = transforms.Compose([
        transforms.Resize((32,32)),
        transforms.ToTensor(),                     
        transforms.Normalize(                     
            mean=[0.485, 0.456, 0.406],               
            std=[0.229, 0.224, 0.225]                  
        )])


#benign data
trafficsign = datasets.ImageFolder(root = 
                             'C:\\Users\Florian\\Desktop\\GTSRB\\Training',
                             transform=data_transform)
original_loader = torch.utils.data.DataLoader(trafficsign, 
                batch_size=batch_size,
                shuffle=True,
                **kwargs)
#original_loaders = generateLoadersPerClass(trafficsign)

#benign test data
testdata = datasets.ImageFolder(root = 
                             'C:\\Users\Florian\\Desktop\\GTSRB\\Test',
                             transform=data_transform)

test_loader = torch.utils.data.DataLoader(testdata, batch_size=batch_size, **kwargs)



# Load backdoor dataset

In [6]:
#malicious data
path = 'C:\\Users\Florian\\Desktop\\GTSRB\\\Training_' + backdoorType
backdoored = datasets.ImageFolder(root = 
                             path,
                             transform=data_transform)
backdoored.samples = [(d, 0) for d, s in backdoored.samples] #set each image of backdoors to 001

backdoored_loader = torch.utils.data.DataLoader(backdoored, 
                batch_size=batch_size,
                shuffle=True,
                **kwargs)
#backdoored_loaders = generateLoadersPerClass(backdoored)

#malicious test data
path = 'C:\\Users\Florian\\Desktop\\GTSRB\\Test_' + backdoorType
backdoored_test = datasets.ImageFolder(root = 
                             path,
                             transform=data_transform)
backdoored_test.samples = [(d, 0) for d, s in backdoored_test.samples] #set each image of backdoors to 001

dataset_loader_backdoored_test = torch.utils.data.DataLoader(backdoored_test, 
                                                             batch_size=batch_size, 
                                                             shuffle=True, **kwargs)

# Visualize some training data

In [7]:
#Let’s visualize a few training images so as to understand the data augmentations.

#def imshow(inp, title=None):
#    """Imshow for Tensor."""
#    inp = inp.numpy().transpose((1, 2, 0))
#    mean = np.array([0.485, 0.456, 0.406])
#    std = np.array([0.229, 0.224, 0.225])
#    inp = std * inp + mean
#    inp = np.clip(inp, 0, 1)
#    plt.imshow(inp)
#    if title is not None:
#        plt.title(title)
#    plt.pause(0.001)  # pause a bit so that plots are updated
#
class_names = trafficsign.classes
## Get a batch of training data
#inputs, classes = next(iter(original_loader))
#
## Make a grid from batch
#out = torchvision.utils.make_grid(inputs)
#
#imshow(out, title=[class_names[x] for x in classes])

# Neural Network Structure

In [15]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv0 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=5) #kernel size = filter size
        self.bn0 = nn.BatchNorm2d(16)
        self.conv1 = nn.Conv2d(16, 32, 5)
        self.bn1 = nn.BatchNorm2d(32)
        self.pool_0 = nn.MaxPool2d(2,stride=2)           #First Max-Pooling Layer
        self.conv2 = nn.Conv2d(32, 96, 3)
        self.bn2 = nn.BatchNorm2d(96)
        self.conv3 = nn.Conv2d(96, 256, 3)
        self.bn3 = nn.BatchNorm2d(256)
        self.pool_1 = nn.MaxPool2d(2, stride=2)
        self.dropout0 = nn.Dropout2d(p=0.37)
        self.fc0 = nn.Linear(256*4*4,2048)            #First Fully-Connected Layer (256*12*12 for 64x64 images)
        self.dropout1 = nn.Dropout2d(p=0.37)
        self.fc1 = nn.Linear(2048, 1024)
        self.dropout2 = nn.Dropout2d(p=0.37)
        self.fc2 = nn.Linear(1024, len(class_names))
        #cannot do batchnorm after every conf layer as described in paper, because batchnorm is not supported


    def forward(self, x):
        #import pdb; pdb.set_trace()
        x = F.relu(self.bn0(self.conv0(x)))
        x = self.pool_0(F.relu(self.bn1(self.conv1(x))))
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.pool_1(F.relu(self.bn3(self.conv3(x))))
        x = self.dropout0(x)
        #print(x.shape)
        x = x.view(-1, 256*4*4)
        x = self.fc0(x)
        x = self.dropout1(x)
        x = self.fc1(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)
    
#http://publications.lib.chalmers.se/records/fulltext/255863/255863.pdf

# Send dataset to clients

In [9]:
train_distributed_dataset = []
train_distributed_dataset_backdoor = []

#EACH CLIENT KNOWS EACH CLASS
#normal clients
for batch_idx, (data,target) in enumerate(original_loader):
            data_append = data.send(compute_nodes[batch_idx % len(compute_nodes)], inplace = True)
            target_append = target.send(compute_nodes[batch_idx % len(compute_nodes)], inplace = True)
            train_distributed_dataset.append((data_append, target_append))

#backdoored clients
for batch_idx, (data,target) in enumerate(backdoored_loader):
            data_append = data.send(frauds[batch_idx % len(frauds)], inplace = True)
            target_append = target.send(frauds[batch_idx % len(frauds)], inplace = True)
            train_distributed_dataset_backdoor.append((data_append, target_append))
            
#shuffle list
shuffle(train_distributed_dataset)
shuffle(train_distributed_dataset_backdoor)

#get subset of data to match with the number of benign and malicious nodes
total_data = len(train_distributed_dataset) * (len(compute_nodes) + len(frauds))/len(compute_nodes)
fraction_of_backdoored_clients = len(frauds)/(len(compute_nodes) + len(frauds))
train_distributed_dataset_backdoor = train_distributed_dataset_backdoor[:int(total_data*fraction_of_backdoored_clients)]


In [10]:
len(train_distributed_dataset)

77

In [11]:
len(train_distributed_dataset_backdoor)

19

# Training Function

In [12]:
def train(epoch, normal_or_backdoored_dataset):
    model.train()
    totalloss = 0
    number_iterations = 0
    
    for batch_idx, (data,target) in enumerate(normal_or_backdoored_dataset):   
        number_iterations +=1
        model.send(data.location) # 0) send the model to the right location
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad() # 1) erase previous gradients (if they exist)
        output = model(data)  # 2) make a prediction
        loss = F.cross_entropy(output, target) # 3) calculate how much we missed
        loss.backward() # 4) figure out which weights caused us to miss
        optimizer.step() # 5) change those weights
        model.get() # 6) get model (with gradients)
            
        #if batch_idx % 300 == 0:
        loss = loss.get() # <-- NEW: get the loss back
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * batch_size, len(normal_or_backdoored_dataset) * batch_size,
                100. * batch_idx / len(normal_or_backdoored_dataset), loss.item()))
        totalloss += loss
    print('Average training loss: {}'.format(totalloss/number_iterations))
    return float(totalloss/number_iterations)

# Test Function

In [13]:
def test(model, device, test_loader, length_of_dataset):
    model.eval()
    test_loss = 0
    correct = 0    
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.cross_entropy(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(1, keepdim=True) # get the index of the max log-probability 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= length_of_dataset

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, length_of_dataset,
        100. * correct / length_of_dataset))
    
    #confusion matrix
    nb_classes = len(class_names)
    confusion_matrix = torch.zeros(nb_classes, nb_classes)
    with torch.no_grad():
        for i, (inputs, classes) in enumerate(test_loader):
            inputs = inputs.to(device)
            classes = classes.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            for t, p in zip(classes.view(-1), preds.view(-1)):
                    confusion_matrix[t.long(), p.long()] += 1
    #print(confusion_matrix)
    per_class_accuracy = confusion_matrix.diag()/confusion_matrix.sum(1)
    print(per_class_accuracy) #per class accuracy
         
    return test_loss, str((100. * correct / length_of_dataset)), per_class_accuracy

# Run everyting

In [14]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01)

#Write to file:
dateString = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

f= open(("exp_gtsrb_"+dateString+".txt"),"w+")
f2= open(("exp_gtsrb_perClassAccuracy_"+dateString+".txt"),"w+")

#EXP-setup
csv_header =  "#dataset: " + "gtsrb" + "\n"
csv_header += "#way backdoor looks like: " + backdoorType + "\n"
csv_header += "#merge strategy: " + "sequential" + "\n"
csv_header += "#number of benign sources: " + str(len(compute_nodes)) + "\n"
csv_header += "#number of malicious sources: " + str(len(frauds)) + "\n"
csv_header += "#batch size: " + str(batch_size) + "\n"
csv_header += "#distribution of data: " + "equally distributed subset" + "\n"
csv_header += "#percentage of poisoned data in backdoored nodes: " + "100" + "\n" #str(100)
csv_header += "#order of time backdoors being inserted: " + "backdoors last" + "\n" #backdoors first
csv_header += "#attack model: " + "basic" + "\n" #basic
csv_header += "#starttime: " + datetime.datetime.now().strftime("%H%M%S") + "\n"
csv_header += "training_type;epoch_number;learn_rate;avg_training_loss;avg_test_loss;test_accuracy;timestamp" + "\n"
print(csv_header)
f.write(csv_header)
f.close()


#RUN training
for epoch in range(1, 101):
    csv_normal = "normal;" + str(epoch) + ";" + str(0.0001) + ";"
    csv_backdoor = "backdoor;" + str(epoch) + ";" + str(0.0001) + ";"
    
    #train normal
    print("Train benign dataset")
    avg_training_loss = train(epoch, train_distributed_dataset)
    csv_normal += str(avg_training_loss) + ";"
    timestamp_normal = datetime.datetime.now().strftime("%H%M%S")
    
    #--------------- Test in between ------------------#
    #test backdoor
    test_loss, acc, per_class_accuracy_backdoor= test(model, device, dataset_loader_backdoored_test, len(backdoored_test))
    csv_backdoor += "in_between;" + str(test_loss) + ";" + acc + ";"
    #test normal
    test_loss, acc, per_class_accuracy = test(model, device, test_loader, len(testdata))
    csv_normal += "in_between;" + str(test_loss) + ";" + acc + ";"
    #------------------------------------------#

    #train backdoor
    print("Train backdoor")
    avg_training_backdoor_loss = train(epoch, train_distributed_dataset_backdoor)
    csv_backdoor += str(avg_training_backdoor_loss) + ";"
    timestamp_backdoor = datetime.datetime.now().strftime("%H%M%S")

    
    #save after each 10 iterations
    if epoch % 10 == 0:
        torch.save(model.state_dict(), ("exp_gtsrb_"+dateString +"_epoch_" + str(epoch) + ".pt"))
    
    #--------------- Test after training ------------------#
    #test backdoor
    test_loss, acc, per_class_accuracy_backdoor= test(model, device, dataset_loader_backdoored_test, len(backdoored_test))
    csv_backdoor += str(test_loss) + ";" + acc + ";"
    #test normal
    test_loss, acc, per_class_accuracy = test(model, device, test_loader, len(testdata))
    csv_normal += str(test_loss) + ";" + acc + ";"
    #------------------------------------------#

    #scheduler.step(test_loss)

    csv_normal += timestamp_normal + "\n"
    csv_backdoor += timestamp_backdoor + "\n"
    
    #Write to file
    f= open(("exp_gtsrb_"+dateString+".txt"),"a+")
    f2=open(("exp_gtsrb_perClassAccuracy_"+dateString+".txt"),"a+")

    f.write(csv_backdoor)
    f.write(csv_normal)
    f2.write(str(per_class_accuracy) + "\n")
    
    f.close()
    f2.close()

#dataset: gtsrb
#way backdoor looks like: backdoor_green_0_5_percent
#merge strategy: sequential
#number of benign sources: 4
#number of malicious sources: 1
#batch size: 512
#distribution of data: equally distributed subset
#percentage of poisoned data in backdoored nodes: 100
#order of time backdoors being inserted: backdoors last
#attack model: basic
#starttime: 090026
training_type;epoch_number;learn_rate;avg_training_loss;avg_test_loss;test_accuracy;timestamp

Train benign dataset
Train Epoch: 1 [0/39424 (0%)]	Loss: 3.759148
Train Epoch: 1 [512/39424 (1%)]	Loss: 3.760089
Train Epoch: 1 [1024/39424 (3%)]	Loss: 3.758306
Train Epoch: 1 [1536/39424 (4%)]	Loss: 3.759228
Train Epoch: 1 [2048/39424 (5%)]	Loss: 3.758211
Train Epoch: 1 [2560/39424 (6%)]	Loss: 3.757757
Train Epoch: 1 [3072/39424 (8%)]	Loss: 3.756695
Train Epoch: 1 [3584/39424 (9%)]	Loss: 3.758461
Train Epoch: 1 [4096/39424 (10%)]	Loss: 3.757013
Train Epoch: 1 [4608/39424 (12%)]	Loss: 3.757737
Train Epoch: 1 [5120/39424 (13%

Train Epoch: 3 [37376/39424 (95%)]	Loss: 3.561591
Train Epoch: 3 [37888/39424 (96%)]	Loss: 3.505019
Train Epoch: 3 [38400/39424 (97%)]	Loss: 3.510128
Train Epoch: 3 [38912/39424 (99%)]	Loss: 3.526485
Average training loss: 3.7205545902252197

Test set: Average loss: 3.6344, Accuracy: 0/12630 (0%)

tensor([0., nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan])

Test set: Average loss: 3.5451, Accuracy: 988/12630 (8%)

tensor([0.0000, 0.3139, 0.7853, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.2403, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000])
Train backdoor
Train Epoch

        nan, nan, nan, nan, nan, nan, nan])

Test set: Average loss: 10.7229, Accuracy: 60/12630 (0%)

tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0.])
Train benign dataset
Train Epoch: 5 [0/39424 (0%)]	Loss: 10.816391
Train Epoch: 5 [512/39424 (1%)]	Loss: 3.589956
Train Epoch: 5 [1024/39424 (3%)]	Loss: 3.575879
Train Epoch: 5 [1536/39424 (4%)]	Loss: 3.604044
Train Epoch: 5 [2048/39424 (5%)]	Loss: 3.577004
Train Epoch: 5 [2560/39424 (6%)]	Loss: 3.619151
Train Epoch: 5 [3072/39424 (8%)]	Loss: 3.608750
Train Epoch: 5 [3584/39424 (9%)]	Loss: 3.616859
Train Epoch: 5 [4096/39424 (10%)]	Loss: 3.581854
Train Epoch: 5 [4608/39424 (12%)]	Loss: 3.623846
Train Epoch: 5 [5120/39424 (13%)]	Loss: 3.596807
Train Epoch: 5 [5632/39424 (14%)]	Loss: 3.568080
Train Epoch: 5 [6144/39424 (16%)]	Loss: 3.582170
Train Epoch: 5 [6656/39424 (17%)]	Loss: 3.594364
Trai

Train Epoch: 7 [38912/39424 (99%)]	Loss: 3.455841
Average training loss: 3.6041135787963867

Test set: Average loss: 3.2289, Accuracy: 0/12630 (0%)

tensor([0., nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan])

Test set: Average loss: 3.4854, Accuracy: 1293/12630 (10%)

tensor([0.0000, 0.5458, 0.7133, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.5069, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000])
Train backdoor
Train Epoch: 7 [0/9728 (0%)]	Loss: 3.235597
Train Epoch: 7 [512/9728 (5%)]	Loss: 2.963834
Train Epoch: 7 [1024/9728 (11%)]	Loss: 2.685457
Train Epoch: 7 [1536/

Train Epoch: 10 [17408/39424 (44%)]	Loss: 3.419296
Train Epoch: 10 [17920/39424 (45%)]	Loss: 3.392554
Train Epoch: 10 [18432/39424 (47%)]	Loss: 3.429990
Train Epoch: 10 [18944/39424 (48%)]	Loss: 3.443420
Train Epoch: 10 [19456/39424 (49%)]	Loss: 3.405122
Train Epoch: 10 [19968/39424 (51%)]	Loss: 3.393548
Train Epoch: 10 [20480/39424 (52%)]	Loss: 3.412758
Train Epoch: 10 [20992/39424 (53%)]	Loss: 3.441256
Train Epoch: 10 [21504/39424 (55%)]	Loss: 3.402185
Train Epoch: 10 [22016/39424 (56%)]	Loss: 3.445090
Train Epoch: 10 [22528/39424 (57%)]	Loss: 3.386822
Train Epoch: 10 [23040/39424 (58%)]	Loss: 3.463333
Train Epoch: 10 [23552/39424 (60%)]	Loss: 3.339067
Train Epoch: 10 [24064/39424 (61%)]	Loss: 3.421662
Train Epoch: 10 [24576/39424 (62%)]	Loss: 3.405242
Train Epoch: 10 [25088/39424 (64%)]	Loss: 3.357321
Train Epoch: 10 [25600/39424 (65%)]	Loss: 3.431399
Train Epoch: 10 [26112/39424 (66%)]	Loss: 3.411259
Train Epoch: 10 [26624/39424 (68%)]	Loss: 3.407504
Train Epoch: 10 [27136/39424 (6

...

           nan,    nan,    nan,    nan,    nan,    nan,    nan])

Test set: Average loss: 0.2632, Accuracy: 11870/12630 (94%)

tensor([0.4667, 0.9639, 0.9813, 0.9600, 0.9348, 0.9175, 0.8067, 0.9089, 0.9689,
        0.9833, 0.9864, 0.9595, 0.9783, 0.9958, 1.0000, 0.9762, 0.9933, 0.9444,
        0.7974, 0.9833, 0.9889, 0.6444, 0.9250, 0.9667, 0.8778, 0.9375, 0.8167,
        0.5000, 0.9600, 0.9667, 0.7333, 0.9815, 0.6667, 0.9905, 0.9917, 0.9641,
        0.9250, 1.0000, 0.9565, 0.7556, 0.8667, 0.8667, 0.8889])
Train backdoor
Train Epoch: 65 [0/9728 (0%)]	Loss: 7.240137
Train Epoch: 65 [512/9728 (5%)]	Loss: 2.461882
Train Epoch: 65 [1024/9728 (11%)]	Loss: 0.155498
Train Epoch: 65 [1536/9728 (16%)]	Loss: 0.099899
Train Epoch: 65 [2048/9728 (21%)]	Loss: 0.045335
Train Epoch: 65 [2560/9728 (26%)]	Loss: 0.038403
Train Epoch: 65 [3072/9728 (32%)]	Loss: 0.024329
Train Epoch: 65 [3584/9728 (37%)]	Loss: 0.020226
Train Epoch: 65 [4096/9728 (42%)]	Loss: 0.015252
Train Epoch: 65 [4608/9728 (47%)]	Loss


tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0.])
Train benign dataset
Train Epoch: 67 [0/39424 (0%)]	Loss: 7.134238
Train Epoch: 67 [512/39424 (1%)]	Loss: 0.406966
Train Epoch: 67 [1024/39424 (3%)]	Loss: 0.314701
Train Epoch: 67 [1536/39424 (4%)]	Loss: 0.284153
Train Epoch: 67 [2048/39424 (5%)]	Loss: 0.229936
Train Epoch: 67 [2560/39424 (6%)]	Loss: 0.200068
Train Epoch: 67 [3072/39424 (8%)]	Loss: 0.240063
Train Epoch: 67 [3584/39424 (9%)]	Loss: 0.213519
Train Epoch: 67 [4096/39424 (10%)]	Loss: 0.182583
Train Epoch: 67 [4608/39424 (12%)]	Loss: 0.184428
Train Epoch: 67 [5120/39424 (13%)]	Loss: 0.169183
Train Epoch: 67 [5632/39424 (14%)]	Loss: 0.188143
Train Epoch: 67 [6144/39424 (16%)]	Loss: 0.150188
Train Epoch: 67 [6656/39424 (17%)]	Loss: 0.172560
Train Epoch: 67 [7168/39424 (18%)]	Loss: 0.165054
Train Epoch: 67 [7680/39424 (19%)]	Loss: 0.

Train Epoch: 69 [33792/39424 (86%)]	Loss: 0.066072
Train Epoch: 69 [34304/39424 (87%)]	Loss: 0.067607
Train Epoch: 69 [34816/39424 (88%)]	Loss: 0.078001
Train Epoch: 69 [35328/39424 (90%)]	Loss: 0.096005
Train Epoch: 69 [35840/39424 (91%)]	Loss: 0.074257
Train Epoch: 69 [36352/39424 (92%)]	Loss: 0.080759
Train Epoch: 69 [36864/39424 (94%)]	Loss: 0.076149
Train Epoch: 69 [37376/39424 (95%)]	Loss: 0.087465
Train Epoch: 69 [37888/39424 (96%)]	Loss: 0.095852
Train Epoch: 69 [38400/39424 (97%)]	Loss: 0.083619
Train Epoch: 69 [38912/39424 (99%)]	Loss: 0.091815
Average training loss: 0.21457646787166595

Test set: Average loss: 7.6185, Accuracy: 100/12630 (1%)

tensor([0.0079,    nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
           nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
           nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
           nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
           nan,  

Train Epoch: 72 [5632/39424 (14%)]	Loss: 0.180521
Train Epoch: 72 [6144/39424 (16%)]	Loss: 0.146656
Train Epoch: 72 [6656/39424 (17%)]	Loss: 0.162746
Train Epoch: 72 [7168/39424 (18%)]	Loss: 0.146585
Train Epoch: 72 [7680/39424 (19%)]	Loss: 0.157812
Train Epoch: 72 [8192/39424 (21%)]	Loss: 0.113913
Train Epoch: 72 [8704/39424 (22%)]	Loss: 0.142196
Train Epoch: 72 [9216/39424 (23%)]	Loss: 0.148297
Train Epoch: 72 [9728/39424 (25%)]	Loss: 0.135097
Train Epoch: 72 [10240/39424 (26%)]	Loss: 0.133062
Train Epoch: 72 [10752/39424 (27%)]	Loss: 0.147441
Train Epoch: 72 [11264/39424 (29%)]	Loss: 0.114457
Train Epoch: 72 [11776/39424 (30%)]	Loss: 0.112548
Train Epoch: 72 [12288/39424 (31%)]	Loss: 0.107556
Train Epoch: 72 [12800/39424 (32%)]	Loss: 0.110248
Train Epoch: 72 [13312/39424 (34%)]	Loss: 0.115811
Train Epoch: 72 [13824/39424 (35%)]	Loss: 0.114612
Train Epoch: 72 [14336/39424 (36%)]	Loss: 0.107224
Train Epoch: 72 [14848/39424 (38%)]	Loss: 0.110247
Train Epoch: 72 [15360/39424 (39%)]	Loss

KeyboardInterrupt: 

In [ ]:
#model = Net().to(device)
#model.load_state_dict(torch.load("newfaces_alexnet_224x224_augmented100.pt"))
#test(model, device, test_loader)


In [16]:
model = Net().to(device)
model

Net(
  (conv0): Conv2d(3, 16, kernel_size=(5, 5), stride=(1, 1))
  (bn0): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv1): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1))
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool_0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 96, kernel_size=(3, 3), stride=(1, 1))
  (bn2): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(96, 256, kernel_size=(3, 3), stride=(1, 1))
  (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool_1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout0): Dropout2d(p=0.37, inplace=False)
  (fc0): Linear(in_features=4096, out_features=2048, bias=True)
  (dropout1): Dropout2d(p=0.37, inplace=False)
  (fc1): Linear(in_features=2048, out_features=1024, bias=True